<a href="https://colab.research.google.com/github/Lkrasnop/Master-degree-Data_science_final_project/blob/data-preparation/Final_Project_EDA_01_10_2023_class_data_preparation_stage_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final project - Lior Krasnopolski and Karin Shauli - class of data preparation

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
from statistics import mean
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.cluster as cluster
from sklearn.cluster import KMeans
import numpy as np

# data preparation class

In [58]:
class data_preparation:
  def __init__(self
               # parameters of our source
               ,our_path_gene_source
               ,number_of_splits
               ,gene_primry_desc
               ,elias_gene_desc
               ,symbol_splitting
               #article parameters
               ,article_path_gen_source
               ,article_gene_primry_desc
               ,article_gene_elias_desc
               ,article_symbol_splitting
               #normalized parameters
               ,normalized_data_path):
    #our gene data set
    self.our_path_gene_source = our_path_gene_source
    self.number_of_splits = number_of_splits
    self.gene_primry_desc = gene_primry_desc
    self.elias_gene_desc = elias_gene_desc
    self.symbol_splitting = symbol_splitting
    #article gene data set
    self.article_gene_primry_desc = article_gene_primry_desc
    self.symbol_splitting_article = article_symbol_splitting
    self.article_path_gen_source = article_path_gen_source
    self.article_gene_elias_desc = article_gene_elias_desc
    #our normalized data set
    self.normalized_data_path = normalized_data_path

  ###perapre our gene data set##############################################
  def our_gene_desc(self):
      numer_of_splits = self.number_of_splits
      df_our_gene_data = pd.read_csv('{}'.format(self.our_path_gene_source))

      # step 1 -  taking only the relavant columns
      df_our_gene_data = df_our_gene_data[['{}'.format(self.gene_primry_desc) ,
                                           '{}'.format(self.elias_gene_desc)]]

      # step 2 - spliting our gene elias###########
      df_our_gene_elias = df_our_gene_data[['{}'.format(self.gene_primry_desc),'{}'.format(self.elias_gene_desc)]].drop_duplicates().reset_index(drop = True)

      # usage of refgene name and spliting by ";" symbol
      df_our_gene_elias = df_our_gene_elias['{}'.format(self.elias_gene_desc)].str.split('{}'.format(self.symbol_splitting), n=numer_of_splits, expand=True)

      # for every elias give sequence column name
      for elem in range(0,len(df_our_gene_elias.columns)):
        df_our_gene_elias.rename(columns={elem:"alias-our-dataframe {}".format(elem+1)},inplace =True)

      df_our_gene_data = pd.concat([df_our_gene_data, df_our_gene_elias], axis=1)

      return  df_our_gene_data


  ###perapre article gene data set##############################################
  def article_gene_source(self):
    # gert of list_sheet_names
    df = pd.ExcelFile('{}'.format(self.article_path_gen_source))
    list_sheet_names = df.sheet_names
    del df

    # get dataframe for gene dataset from aritcle
    data_frame_gene = []

    for name in list_sheet_names:
      temp = pd.read_excel('{}'.format(self.article_path_gen_source) , sheet_name='{}'.format(name))
      temp = temp[['{}'.format(self.article_gene_primry_desc),'{}'.format(self.article_gene_elias_desc)]]
      temp['group_name'] = name
      data_frame_gene.append(temp)

    data_frame_gene = pd.concat(data_frame_gene)

    #split the alias
    numer_of_splits = self.number_of_splits
    split_alias = data_frame_gene['{}'.format(self.article_gene_elias_desc)].str.split('{}'.format(self.symbol_splitting_article), n=numer_of_splits, expand=True)

    #change names of added columns
    for elem in range(0,len(split_alias.columns)):
      split_alias.rename(columns={elem:"alias {}".format(elem+1)},inplace =True)

    #merge alias splited dataframe into gene frame
    data_frame_gene = pd.concat([data_frame_gene, split_alias], axis=1)

    #drop alias inital column
    data_frame_gene = data_frame_gene.drop(['{}'.format(self.article_gene_elias_desc)], axis = 1)

    return data_frame_gene

  ###get normalized data for each patient##############################################
  def genes_from_article_source_isin_genes_from_our_source(self):
    data_frame_article_gene_source = self.article_gene_source()
    data_frame_our_gene_source = self.our_gene_desc()

    # two for loops that check if each elias in article data frame appears in our data gene elias
    for column_our in data_frame_our_gene_source:
        for column_article in data_frame_article_gene_source:
            data_frame_gene = data_frame_our_gene_source.loc[data_frame_our_gene_source[column_our].isin(data_frame_article_gene_source[column_article])]

    return data_frame_gene



  ###get normalized data for each patient##############################################
  def get_normalized_data(self):
    df_normalized_celltype_reconstruct  = pd.read_csv('{}'.format(self.normalized_data_path))
    df_normalized_celltype_reconstruct = df_normalized_celltype_reconstruct.drop(columns = ['Unnamed: 0'] , axis=1)
    return df_normalized_celltype_reconstruct





# initiate parameters

In [59]:
#our gene source - parameters
our_gene_path = '/content/drive/MyDrive/Final Project Data Science Master/קבצים ישנים/all_450K.csv'
number_of_splits = 13
gene_primry_desc = 'Name'
elias_gene_desc = 'UCSC_RefGene_Name'
symbol_splitting = ';'

#article - parameters
article_path_gen_source = '/content/drive/MyDrive/Final Project Data Science Master/ 41419_2021_4121_MOESM1_ESM_cleaned.xlsx'
article_gene_primry_desc = 'Official Gene symbol'
article_gene_elias_desc = 'Alias (optional)'
symbol_splitting_article = ','

#normalized data parameters
normalized_data_path = '/content/drive/MyDrive/Final Project Data Science Master/df_GSE87571_normalized_celltype_reconstruct.csv'


data_preparation = data_preparation(our_gene_path,
                                    number_of_splits,
                                    gene_primry_desc,
                                    elias_gene_desc,
                                    symbol_splitting,
                                    article_path_gen_source,
                                    article_gene_primry_desc,
                                    article_gene_elias_desc,
                                    symbol_splitting_article,
                                    normalized_data_path)

# import data

In [ ]:
our_data_before = data_preparation.our_gene_desc()
our_data_after = data_preparation.genes_from_article_source_isin_genes_from_our_source()

In [76]:
print('befor cleaning irrelevant genes: ' + f'{our_data_before.UCSC_RefGene_Name.nunique()}')
print('after cleaning irrelevant genes: '+ f'{our_data_after.UCSC_RefGene_Name.nunique()}')

befor cleaning irrelevant genes: 42455
after cleaning irrelevant genes: 42224


In [68]:
(1-((our_data_before.UCSC_RefGene_Name.nunique()- our_data_after.UCSC_RefGene_Name.nunique())/our_data_before.UCSC_RefGene_Name.nunique()))*100

99.45589447650454

In [53]:
our_data = data_preparation.genes_from_article_source_isin_genes_from_our_source()
our_data

<ipython-input-49-47792b8a53bc>:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['group_name'] = name
<ipython-input-49-47792b8a53bc>:35: DtypeWarning: Columns (2,4,11,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_our_gene_data = pd.read_csv('{}'.format(self.our_path_gene_source))


,Name,UCSC_RefGene_Name,alias-our-dataframe 1,alias-our-dataframe 2,alias-our-dataframe 3,alias-our-dataframe 4,alias-our-dataframe 5,alias-our-dataframe 6,alias-our-dataframe 7,alias-our-dataframe 8,alias-our-dataframe 9,alias-our-dataframe 10,alias-our-dataframe 11,alias-our-dataframe 12,alias-our-dataframe 13,alias-our-dataframe 14
0,cg00035864,TTTY18,TTTY18,None,None,None,None,None,None,None,None,None,None,None,None,None
1,cg00050873,TSPY4;FAM197Y2,TSPY4,FAM197Y2,None,None,None,None,None,None,None,None,None,None,None,None
2,cg00061679,DAZ1;DAZ4;DAZ4,DAZ1,DAZ4,DAZ4,None,None,None,None,None,None,None,None,None,None,None
3,cg00063477,EIF1AY,EIF1AY,None,None,None,None,None,None,None,None,None,None,None,None,None
4,cg00121626,BCORL2,BCORL2,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486423,NORM_T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
486424,NORM_C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
486425,NORM_T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
486426,NORM_C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
our_data.UCSC_RefGene_Name.nunique()

42455

# import normalized celltype dataframe

In [ ]:
df_GSE87571_normalized_celltype_reconstruct  = pd.read_csv('/content/drive/MyDrive/Final Project Data Science Master/df_GSE87571_normalized_celltype_reconstruct.csv')
df_GSE87571_normalized_celltype_reconstruct = df_GSE87571_normalized_celltype_reconstruct.drop(columns = ['Unnamed: 0'] , axis=1)
df_GSE87571_normalized_celltype_reconstruct

,source_name,cg,normalized
0,GSM2333902,cg00000108,0.991673
1,GSM2333903,cg00000108,0.993019
2,GSM2333901,cg00000108,0.992090
3,GSM2333905,cg00000108,0.993610
4,GSM2333908,cg00000108,0.987779
...,...,...,...
97070767,GSM2334627,cg27666123,0.963326
97070768,GSM2334630,cg27666123,0.968636
97070769,GSM2334631,cg27666123,0.944395
97070770,GSM2334632,cg27666123,0.933849


# taking only the relevant cases : filtering only the relevant probes and patients.

In [ ]:
# drop all irrelevant patients and cgs
df_GSE87571_normalized_celltype_reconstruct = df_GSE87571_normalized_celltype_reconstruct.loc[df_GSE87571_normalized_celltype_reconstruct.source_name.isin(df_info.source_name)]
df_GSE87571_normalized_celltype_reconstruct = df_GSE87571_normalized_celltype_reconstruct.loc[df_GSE87571_normalized_celltype_reconstruct.cg.isin(df_our_gene_data.Name)]
df_GSE87571_normalized_celltype_reconstruct = df_GSE87571_normalized_celltype_reconstruct.dropna()
display(df_GSE87571_normalized_celltype_reconstruct)

,source_name,cg,normalized
831,GSM2333902,cg00000321,0.267138
832,GSM2333903,cg00000321,0.286995
833,GSM2333901,cg00000321,0.329600
834,GSM2333905,cg00000321,0.318397
835,GSM2333908,cg00000321,0.260389
...,...,...,...
97070490,GSM2334627,cg27666046,0.553640
97070491,GSM2334630,cg27666046,0.606716
97070492,GSM2334631,cg27666046,0.543427
97070493,GSM2334632,cg27666046,0.596561


# merged between normalized (probes) dataframe and pateints' general info


In [ ]:
#merge between normalized dataframe and the patients'info
df_GSE87571_normalized_celltype_and_patient_info= pd.merge(df_GSE87571_normalized_celltype_reconstruct,
                     df_info,
                     on=['source_name'],
                     how='left')
del df_GSE87571_normalized_celltype_reconstruct , df_info

In [ ]:
#merge between normalized dataframe and the genes' info
df_GSE87571_normalized_celltype_and_patient_info= pd.merge(df_GSE87571_normalized_celltype_and_patient_info,
                     df_our_gene_data,
                     right_on=['Name'],
                     left_on=['cg'],
                     how='left')
del df_our_gene_data

### drop irrelevant columns

In [ ]:
df_GSE87571_normalized_celltype_and_patient_info = df_GSE87571_normalized_celltype_and_patient_info[['cg','normalized','age','UCSC_RefGene_Name']]
df_GSE87571_normalized_celltype_and_patient_info.rename(columns={'UCSC_RefGene_Name':"gene"},inplace =True)
display(df_GSE87571_normalized_celltype_and_patient_info)

<ipython-input-24-ebd69ee66860>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_GSE87571_normalized_celltype_and_patient_info.rename(columns={'UCSC_RefGene_Name':"gene"},inplace =True)


,cg,normalized,age,gene
0,cg00000321,0.267138,55.0,SFRP1
1,cg00000321,0.286995,23.0,SFRP1
2,cg00000321,0.329600,72.0,SFRP1
3,cg00000321,0.318397,74.0,SFRP1
4,cg00000321,0.260389,38.0,SFRP1
...,...,...,...,...
63296157,cg27666046,0.553640,53.0,SECTM1
63296158,cg27666046,0.606716,70.0,SECTM1
63296159,cg27666046,0.543427,72.0,SECTM1
63296160,cg27666046,0.596561,54.0,SECTM1


# saving the final dataframe

In [ ]:
df_GSE87571_normalized_celltype_and_patient_info = pd.read_csv('/content/drive/MyDrive/Final Project Data Science Master/df_final.csv')
df_GSE87571_normalized_celltype_and_patient_info = df_GSE87571_normalized_celltype_and_patient_info[['cg','normalized','age','UCSC_RefGene_Name']]
df_GSE87571_normalized_celltype_and_patient_info.rename(columns={'UCSC_RefGene_Name':"gene"},inplace =True)
display(df_GSE87571_normalized_celltype_and_patient_info)

<ipython-input-3-cd9f96fd06a4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_GSE87571_normalized_celltype_and_patient_info.rename(columns={'UCSC_RefGene_Name':"gene"},inplace =True)


,cg,normalized,age,gene
0,cg00000321,0.267138,55.0,SFRP1
1,cg00000321,0.286995,23.0,SFRP1
2,cg00000321,0.329600,72.0,SFRP1
3,cg00000321,0.318397,74.0,SFRP1
4,cg00000321,0.260389,38.0,SFRP1
...,...,...,...,...
63296157,cg27666046,0.553640,53.0,SECTM1
63296158,cg27666046,0.606716,70.0,SECTM1
63296159,cg27666046,0.543427,72.0,SECTM1
63296160,cg27666046,0.596561,54.0,SECTM1


In [ ]:
df_GSE87571_normalized_celltype_and_patient_info.to_csv('/content/drive/MyDrive/Final Project Data Science Master/df_final.csv')